In [1]:
#----------------------------------------------------------------------------#
#       Procesamiento de grandes volumenes de datos 2020-2                   #
#               Proyecto 1 (data cleaning + MLlib)                           #
#                       Alejandro Ayala Gil                                  #
#                       Esteban Cardona Gil                                  #
#                    Juan Camilo Gomez Muñoz                                 #
#                        Julian Paredes C                                    #
#                    Tania C. Obando Suárez                                  #
#----------------------------------------------------------------------------#




if('__name__' == '__main__'):
    #Importando librerias
    from pyspark import SparkConf, SparkContext
    from pyspark.sql import SQLContext, Row
    from pyspark.sql import SparkSession, DataFrameStatFunctions, DataFrameNaFunctions
    from pyspark.sql.functions import *
    #import seaborn as sns
    import matplotlib.pyplot as plt
    from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
    from pyspark.ml.classification import LinearSVC
    from pyspark.ml.feature import VectorAssembler
    from pyspark.ml.evaluation import BinaryClassificationEvaluator
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    #Encabezado del dataframe
    headings = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob',
         'Fjob','reason','guardian','traveltime','studytime','failures','schoolsup',
         'famsup','paid','activities','nursery','higher','internet','romantic',
         'famrel','freetime','goout','Dalc','Walc','health','absences','G1','G2','G3']
    spark = SparkSession.builder.appName("Student").getOrCreate()

    #Crear dataframe

    esquema=spark.read.csv('student-por.csv',sep=';',header=True).schema
    df=spark.readStream.csv('student-por.csv',sep=';',header=True,schema=esquema)
    print("Hola mundo")

In [ ]:
StructType(List(StructField(school,StringType,true),StructField(sex,StringType,true),StructField(age,StringType,true),StructField(address,StringType,true),StructField(famsize,StringType,true),StructField(Pstatus,StringType,true),StructField(Medu,StringType,true),StructField(Fedu,StringType,true),StructField(Mjob,StringType,true),StructField(Fjob,StringType,true),StructField(reason,StringType,true),StructField(guardian,StringType,true),StructField(traveltime,StringType,true),StructField(studytime,StringType,true),StructField(failures,StringType,true),StructField(schoolsup,StringType,true),StructField(famsup,StringType,true),StructField(paid,StringType,true),StructField(activities,StringType,true),StructField(nursery,StringType,true),StructField(higher,StringType,true),StructField(internet,StringType,true),StructField(romantic,StringType,true),StructField(famrel,StringType,true),StructField(freetime,StringType,true),StructField(goout,StringType,true),StructField(Dalc,StringType,true),StructField(Walc,StringType,true),StructField(health,StringType,true),StructField(absences,StringType,true),StructField(G1,StringType,true),StructField(G2,StringType,true),StructField(G3,StringType,true)))